## PURPOSE OF THE NOTEBOOK + TITLE OF THE PROJECT

In [2]:
import requests
from requests.adapters import HTTPAdapter
import json

import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
import time
import numpy as np
import re

In [ ]:
links = ['https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=1','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=2','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=3','https://www.ravelry.com/patterns/search#pc=medical&sort=name&view=captioned_thumbs&page=4']

### Get the ID's of all the recipes in the medical knitting recipe category

In [ ]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless = False)
page = await browser.new_page()

url = links[0]
await page.goto(url)

# Use playwright to login to the site. The xxxxx replace the username and password:
await page.locator("#user_login").fill('xxxxx')
await page.locator("#user_password").fill('xxxxx')
await page.locator("#login_form > div.splash_page__submit > div.splash_page__submit__login > button").click()

time.sleep(5)

pattern_ids = []

# Scrape each of the four pages:
for link in links:
    await page.goto(link)
    time.sleep(5)
    html = await page.content()
    doc = BeautifulSoup(html)

    content_card = doc.find_all(class_= "pattern captioned_pattern captioned_search_result")

    pattern_id = []

    # Loop through each pattern on the page and get the information:
    for card in content_card:
        pattern = {}
        pattern['name'] = card.find('a').get('title')
        pattern['url'] = card.find('a').get('href')

        find_id = card.get('id')
        pattern['id'] = re.findall(r'\d+', find_id)[0]

        # And append the pattern data to the list:
        pattern_id.append(pattern)
    pattern_ids.append(pattern_id)

### And take the list of lists (with some dictionaries inside) created in the scrape and turn it into one flat list.

This allows me to turn it into a dataframe and then into a csv file.

In [ ]:
flat_list = []
for ids in pattern_ids:
    for id in ids:
        flat_list.append(id)

id_df = pd.DataFrame(flat_list)

In [ ]:
id_df.to_csv("data/patterns.csv", index=False)

## Get information on the patterns

In [ ]:
df = pd.read_csv("data/patterns.csv")

In [ ]:
url_list = df.url.values.tolist()

In [ ]:
project_numbers = []

for url in url_list:
    await page.goto(url)

    time.sleep(0.5)

    project = {}

    html = await page.content()
    doc = BeautifulSoup(html)

    type = doc.find_all(class_='value')
    type = type[1].text
    type = re.sub(r'\n', '', type)

    project['craft_type'] = type
    
    projects = doc.find(class_= "pattern_people summary_box_people").text
    # replace \n\n\n with nothing
    projects = re.sub(r'\n\n\n', '', projects)
    project['project_numbers'] = re.sub(r' projec\D*,\nin \d* queues\n\n', '', projects)
    
    project_numbers.append(project)
await browser.close()

In [ ]:
df = pd.DataFrame(project_numbers)
df.to_csv("data/patterns_info.csv", index=False)

## WHAT IS HAPPENING HERE

In [ ]:
patterns = pd.read_csv("data/patterns.csv")
patterns_info = pd.read_csv("data/patterns_info.csv")

In [ ]:
# Merge the two dataframes using concat
total = pd.concat([patterns, patterns_info], axis=1)

# Give the recipe with no projects the value 0
total = total.replace(regex=r'no projects\n', value=0)

In [ ]:
# And save it as a csv
total.to_csv("data/patterns_total.csv", index=False)

For the next part I have to manually categorize the recipes.

In [5]:
df = pd.read_csv("data/patterns_total_categorized.csv")

,name,url,id,craft_type,project_numbers,Category
0,"""Hey, at Least It Wasn't Your Leg"" Arm Sling b...",https://www.ravelry.com/patterns/library/hey-a...,80882,Knitting,5,injury
1,"""Kitty"" Eye Patch by Christine H. Wilkins",https://www.ravelry.com/patterns/library/kitty...,118789,Knitting,5,injury
2,Alternative ML Knitted Knockers by Makoshark,https://www.ravelry.com/patterns/library/alter...,951015,Knitting,4,knockers
3,Amon ammeinnlegg by Anne Tvedt,https://www.ravelry.com/patterns/library/amon-...,807316,Knitting,0,maternity
4,Amputee's Stump Covers by Suzetta Williams,https://www.ravelry.com/patterns/library/amput...,287776,Crochet,12,amputee
...,...,...,...,...,...,...
165,Woolly Boob by Kathryn Cottam,https://www.ravelry.com/patterns/library/wooll...,639320,Crochet,1,maternity
166,Worry Bear by Joanne Seiff,https://www.ravelry.com/patterns/library/worry...,908676,Knitting,3,mental_psycological
167,Wrapped in Lace PICC Line Cover by Kirsten Hol...,https://www.ravelry.com/patterns/library/wrapp...,749754,Crochet,9,cannula_iv
168,Wrist rest by Lynne Rowe,https://www.ravelry.com/patterns/library/wrist...,464528,Knitting,3,pain_relief


What is the most popular category?

In [26]:
categories = df.groupby('Category').agg({'project_numbers':'sum'}).reset_index().sort_values(by='project_numbers', ascending=False)
categories.drop(categories[categories['Category'] == 'na'].index, inplace = True)

categories = categories.head(5)
categories

,Category,project_numbers
6,knockers,4744
4,heating_cooling,261
5,injury,252
8,menstruation,196
1,cannula_iv,58


In [25]:
categories.to_csv("data/categories.csv", index=False)

Are knockers most often knitted or croched?

In [13]:
knockers = df[df['Category'] == 'knockers']
knockers_grouped = knockers.groupby('craft_type').agg({'project_numbers':'sum'}).reset_index().sort_values(by='project_numbers', ascending=False)

In [33]:
knockers_grouped['percent'] = knockers_grouped.project_numbers / knockers_grouped.project_numbers.sum() * 100
knockers_grouped = knockers_grouped.round(0)
knockers_grouped

,craft_type,project_numbers,percent
1,Knitting,4441,94.0
0,Crochet,301,6.0
2,Loom Knitting,2,0.0


In [32]:
knockers_grouped.to_csv("data/knockers_grouped.csv", index=False)

Use the Ravelry API to get information about projects that uses each of the recipes in the medical knitting recipe category.

In [ ]:
pattern_info = []
for id in col_list:
    api_key_id = 'xxxxxx'
    api_key_secret = 'xxxxxx'
    headers = {'Content-Type': 'application/json'}
    auth = (api_key_id, api_key_secret)
    newSession = requests.Session()
    link = f"https://api.ravelry.com/patterns/{id}/projects.json"
    response = newSession.post('https://api.ravelry.com/patterns/80882/projects.json',auth=auth,headers=headers,verify=False) 
    print(response.status_code)
    data = response.json()
    pattern_info.append(data)

In [ ]:
api_key_id = 'xxxxxx'
api_key_secret = 'xxxxxx'
headers = {'Content-Type': 'application/json'}
auth = (api_key_id, api_key_secret)
newSession = requests.Session()
response = newSession.post('https://api.ravelry.com/patterns/search.json?category=medical',auth=auth,headers=headers,verify=False) 
print(response.status_code)

In [ ]:
data = response.json()

In [ ]:

api_key_id = 'xxxxxx'
api_key_secret = 'xxxxxx'
headers = {'Content-Type': 'application/json'}
auth = (api_key_id, api_key_secret)
newSession = requests.Session()
response = newSession.post('https://api.ravelry.com/patterns/503114/projects.json?page_size=5000',auth=auth,headers=headers,verify=False) 
print(response.status_code)
pattern_data = response.json()
len(pattern_data['projects'])
